# Carregamento e pré-processamento dos dados

https://www.kaggle.com/datasets/denkuznetz/food-delivery-time-prediction

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("denkuznetz/food-delivery-time-prediction")

print("Path to dataset files:", path)

In [ ]:
import os

# Listar os arquivos no diretório
files = os.listdir(path)
print("Arquivos no diretório:", files)

In [ ]:
import pandas as pd

In [ ]:
csv_file_path = os.path.join(path, 'Food_Delivery_Times.csv')

df = pd.read_csv(csv_file_path)
df.head()

In [ ]:
from sklearn.impute import SimpleImputer

# Remover a coluna Order_ID
df.drop(columns=['Order_ID'], inplace=True)

# Substituir valores nulos em "Courier_Experience_yrs" pela mediana
imputer_num = SimpleImputer(strategy="median")
df["Courier_Experience_yrs"] = imputer_num.fit_transform(df[["Courier_Experience_yrs"]])

# Substituir valores nulos em variáveis categóricas pela moda
categorical_cols = ["Weather", "Traffic_Level", "Time_of_Day"]
imputer_cat = SimpleImputer(strategy="most_frequent")
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])

# Aplicar One-Hot Encoding nas variáveis categóricas
df = pd.get_dummies(df, columns=["Weather", "Traffic_Level", "Time_of_Day", "Vehicle_Type"], drop_first=True)

# Exibir as primeiras linhas do dataframe processado
df.head()


In [ ]:
for col in df.columns:
    if df[col].dtype == 'object':
        # If column is of object type, try converting to numeric
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            # If conversion fails, handle the non-numeric values (e.g., replace with NaN or a specific value)
            # Here, we replace non-numeric values with NaN and then fill them with the column's median
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert non-numeric to NaN
            df[col] = df[col].fillna(df[col].median())  # Fill NaN with median

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Delivery_Time_min']).values
y = df['Delivery_Time_min'].values
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Selecionando as três primeiras colunas (dados contínuos)
continuous_indices = [0, 1, 2]

# Aplicando MinMaxScaler
scaler = MinMaxScaler()
X_train[:, continuous_indices] = scaler.fit_transform(X_train[:, continuous_indices])
X_test[:, continuous_indices] = scaler.transform(X_test[:, continuous_indices])

In [ ]:
# Normalizando o alvo
y_min = y_train.min()
y_max = y_train.max()

y_train = (y_train - y_min) / (y_max - y_min)
y_test = (y_test - y_min) / (y_max - y_min)


In [ ]:
import numpy as np
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# O problema

!['Moto'](https://i.imgur.com/kGUQ2wj.png)

Imagine uma cidade grande e agitada, onde os moradores estão sempre em movimento e dependem de serviços de entrega rápida para suas refeições. A **Run Food Run**, uma empresa de entregas por aplicativo, enfrenta diariamente o desafio de garantir que seus motoboys entreguem pedidos no menor tempo possível, mantendo a qualidade do serviço.

No entanto, prever o tempo de entrega não é uma tarefa simples. Diversos fatores influenciam esse processo: o clima, o horário do dia, a distância entre o restaurante e o cliente, o tráfego nas ruas, e até mesmo o histórico de desempenho dos motoboys. A empresa percebeu que, para continuar competitiva, precisava de uma solução tecnológica que ajudasse a prever os tempos de entrega com maior precisão.

E é aqui que entra o poder das **redes neurais**. Utilizando dados históricos de entregas, como características das rotas e tempos registrados, a Run Food Run decidiu criar um modelo de regressão baseado em aprendizado de máquina. O objetivo? Prever com precisão o tempo que um motoboy levará para entregar um pedido, antes mesmo de ele sair do restaurante.

Com isso, a empresa poderia:
- **Aumentar a eficiência**: Alocar os motoboys de forma mais estratégica, reduzindo atrasos.
- **Melhorar a experiência do cliente**: Fornecer previsões mais precisas sobre o tempo de entrega.
- **Otimizar recursos**: Identificar padrões que poderiam ser ajustados para melhorar os tempos médios de entrega.

Neste projeto, você será transportado para o coração desse desafio. Usando o **PyTorch**, uma das ferramentas mais poderosas para aprendizado de máquina, você irá realizar a construção de uma rede neural capaz de transformar dados brutos em previsões úteis e práticas.


# Tarefa 1: Convertendo Dados para Tensores do PyTorch

O primeiro passo para trabalharmos com o PyTorch é converter os dados de treino e teste (`X_train`, `X_test`, `y_train` e `y_test`) em **tensores**, que são a estrutura de dados fundamental no PyTorch. Os tensores permitem que o PyTorch realize operações matemáticas otimizadas, aproveitando o poder de GPUs para acelerar os cálculos.

Sua tarefa é:
1. Converter os conjuntos de entrada (`X_train` e `X_test`) e saída (`y_train` e `y_test`) em tensores do PyTorch.
2. Garantir que os tensores de saída (`y_train` e `y_test`) tenham a forma correta para serem usados no modelo (dica: você pode usar `.view(-1, 1)` para ajustar a dimensão).

#### Dica:
Você pode usar a função `torch.tensor()` para realizar a conversão. Não se esqueça de especificar o tipo de dado como `torch.float32`, pois o PyTorch trabalha melhor com esse formato em problemas de aprendizado de máquina.

https://pytorch.org/docs/stable/tensors.html

# Tarefa 2: Criando o Dataset e o DataLoader

Agora que os dados foram convertidos para tensores, o próximo passo é organizá-los de forma eficiente para serem usados durante o treinamento do modelo. Para isso, utilizaremos as classes **`Dataset`** e **`DataLoader`** do PyTorch.

#### Sua tarefa:
1. **Criar um Dataset personalizado**: Use a classe `torch.utils.data.Dataset` para criar um conjunto de dados que armazene os tensores de entrada (`X_train_tensor`, `X_test_tensor`) e saída (`y_train_tensor`, `y_test_tensor`).
2. **Criar DataLoaders**: Use a classe `DataLoader` para carregar os dados em lotes (*batches*), o que é essencial para treinar modelos de forma eficiente.

#### Dicas:
- Para criar o Dataset, você precisará implementar os métodos `__len__` (que retorna o tamanho do dataset) e `__getitem__` (que retorna uma amostra específica do dataset com base no índice).
- O DataLoader permite que você carregue os dados em lotes e, no caso do conjunto de treino, pode embaralhá-los para melhorar o treinamento.

Consultar o tópico: Creating a Simple Dataset Class
https://machinelearningmastery.com/using-dataset-classes-in-pytorch/

Criar DataLoaders:
Os DataLoaders permitem carregar os dados em lotes (batches) e embaralhá-los durante o treinamento.

https://www.geeksforgeeks.org/how-to-use-a-dataloader-in-pytorch/

# Tarefa 3: Definindo o Modelo

Agora que os dados estão organizados e prontos para uso, o próximo passo é definir o modelo de rede neural. Para isso, utilizaremos o **PyTorch** para criar uma arquitetura simples, mas poderosa, capaz de realizar previsões em um problema de regressão.

#### Sua tarefa:
1. **Definir a arquitetura do modelo**: Crie uma classe que herda de `torch.nn.Module` e implemente as camadas da rede neural.
2. **Configurar as camadas**: Para um problema de regressão, a última camada deve ter **1 unidade de saída** (saída linear, sem função de ativação).
3. **Adicionar funções de ativação**: Use funções de ativação como `ReLU` entre as camadas ocultas para introduzir não-linearidades.

#### Dicas:
- Use a classe `nn.Sequential` para organizar as camadas de forma simples.
- A arquitetura pode começar com uma camada de entrada (tamanho igual ao número de *features*), seguida por algumas camadas ocultas, e terminar com uma camada de saída.

#### Tarefa prática:
1. Defina o modelo usando a classe `torch.nn.Module`.
2. Certifique-se de que o número de entradas da primeira camada (`input_size`) seja igual ao número de *features* do seu conjunto de dados.
3. Inicialize o modelo e verifique se ele está pronto para receber entradas.

#### Dica extra:
Para inicializar o modelo, você pode usar:
```python
input_size = X_train_tensor.shape[1]  # Número de *features* no conjunto de treino
model = RegressionModel(input_size)
```


https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html

# Tarefa 4: Definir a função de custo e o otimizador

Com o modelo definido, é hora de configurar os elementos essenciais para o treinamento: a **função de custo** (ou função de perda) e o **otimizador**. Esses componentes irão guiar o modelo durante o aprendizado, ajustando os pesos para minimizar os erros nas previsões.

#### Sua tarefa:
1. **Escolher a função de custo**: Para um problema de regressão, uma boa escolha é o **Erro Quadrático Médio (MSE)**, que mede a diferença entre as previsões do modelo e os valores reais.
2. **Definir o otimizador**: O otimizador é responsável por ajustar os pesos do modelo com base no gradiente da função de custo. O **Adam** é uma escolha popular e eficiente.

#### Dicas:
- A função de custo pode ser definida usando `torch.nn.MSELoss()`.
- O otimizador pode ser configurado com `torch.optim.Adam()`, passando os parâmetros do modelo (`model.parameters()`) e a taxa de aprendizado (`learning_rate`).

https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html


https://pytorch.org/docs/stable/optim.html


# Tarefa 5: Treinar o modelo

Agora que o modelo, a função de custo e o otimizador estão configurados, é hora de realizar o treinamento! O objetivo aqui é ajustar os pesos do modelo para minimizar o erro nas previsões, utilizando os dados de treino.

#### Tarefa prática:
1. Configure o número de *épocas* (ex.: `num_epochs = 100`).
2. Use o `DataLoader` para iterar sobre os lotes do conjunto de treino.
3. Para cada lote:
   - Faça a previsão (`forward pass`).
   - Calcule a perda.
   - Realize o retropropagação (`backward pass`) e atualize os pesos.
4. Monitore a perda ao longo das épocas para garantir que o modelo está aprendendo.

#### Dica extra:
- Você pode adicionar validação ao final de cada época, utilizando o `test_loader` para verificar o desempenho do modelo nos dados de teste.

Consultar Training Loop:
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html


# Tarefa 6: Avaliar o modelo

Após o treinamento do modelo, é essencial avaliar seu desempenho para entender como ele se comporta nos dados de teste (ou validação). Isso ajuda a garantir que o modelo está generalizando bem e não apenas "decorando" os dados de treino.

#### Sua tarefa:
1. **Colocar o modelo em modo de avaliação**: Use `model.eval()` para desativar componentes como *dropout* (caso existam) e garantir que o modelo funcione de forma determinística.
2. **Calcular métricas de desempenho**: Para problemas de regressão, a métrica mais comum é o **Erro Quadrático Médio (MSE)** ou o **Erro Absoluto Médio (MAE)**. Você também pode calcular o **R² (coeficiente de determinação)** para avaliar o ajuste geral.
3. **Evitar gradientes durante a avaliação**: Use `torch.no_grad()` para evitar o cálculo de gradientes, o que economiza memória e acelera a avaliação.

https://yassin01.medium.com/understanding-the-difference-between-model-eval-and-model-train-in-pytorch-48e3002ee0a2

# Tarefa 7: Fazer previsões